# Bell State Preparation experiment: Compute $\langle \bar{\Phi^+} | \bar{X}\bar{X} | \bar{\Phi^+}\rangle$ with the seven-qubit Steane code

In [1]:
from typing import List, Dict, Sequence
import itertools
import functools
import numpy as np
from tqdm import tqdm
import cirq
import qiskit
from qiskit.circuit.library import Barrier
import qiskit_ibm_runtime
from qiskit_aer import AerSimulator

from mitiq import PauliString

from encoded.diagonalize import get_stabilizer_matrix_from_paulis, get_measurement_circuit, get_paulis_from_stabilizer_matrix

## Set parameters

In [2]:
n = 7                                   # Number of physical qubits
nshots = 100_000                        # Number of samples/shots
depth = 0                               # Number of folded Bell state preparation circuits for added noise
k = 2                                   # Number of logical qubits.


In [15]:
# Computer and qubits to use.
# Option 1: Use saved noise characteristics.
#computer = qiskit_ibm_runtime.fake_provider.FakeKyiv()
computer = AerSimulator()  # Noiseless simulator - use this for a sanity check to see all computed expectation values (physical and encoded) are 1.0.

# Option 2: Use noise characteristics from most recent calibration.
# service = qiskit_ibm_runtime.QiskitRuntimeService()  # This assumes a saved account.
# computer = service.backend("ibm_kyiv")
# computer = AerSimulator.from_backend(computer)

# See calibration data at https://quantum.ibm.com/services/resources to select good qubits.
layout = {
    1 : [0,1],
    7 : [61, 62, 63, 64, 65, 66, 67, 72, 73, 81, 82, 83, 84, 85],
}
layout = {
1 : [0,1],
7 : [0,1,2,3,4,5,6,8,9,10,11,12,13],

}

## Helper functions

In [16]:
# Expectation of pauli on bitstring measured in diagonal basis.
def compute_expectation(
    pauli: cirq.PauliString,
    counts: Dict[str, int],
) -> float:
    if pauli is cirq.PauliString():
        return 1.0

    expectation = 0.0

    indices = [q.x for q in pauli.qubits]
    for key, value in counts.items():
        key = list(map(int, list(key[::-1])))
        expectation += (-1) ** sum([key[i] for i in indices]) * value

    return expectation / sum(counts.values())

# Prepares logical |0> state on Steane Code
def encode_steane(qreg: Sequence[cirq.Qid]) -> cirq.Circuit:
    circuit = cirq.Circuit()

    circuit.append(cirq.H.on(qreg[0]))
    circuit.append(cirq.H.on(qreg[4]))
    circuit.append(cirq.H.on(qreg[6]))

    circuit.append(cirq.CNOT.on(qreg[0], qreg[1]))
    circuit.append(cirq.CNOT.on(qreg[4], qreg[5]))

    circuit.append(cirq.CNOT.on(qreg[6], qreg[3]))
    circuit.append(cirq.CNOT.on(qreg[6], qreg[5]))
    circuit.append(cirq.CNOT.on(qreg[4], qreg[2]))
    
    circuit.append(cirq.CNOT.on(qreg[0], qreg[3]))
    circuit.append(cirq.CNOT.on(qreg[4], qreg[1]))
    circuit.append(cirq.CNOT.on(qreg[3], qreg[2]))

    return circuit

def strs_to_paulis(pauli_strs : List[str]) -> List[cirq.PauliString]:
    stab_list = []
    for stab_str in pauli_strs:
        stab_list.append(PauliString(stab_str)._pauli)
    return stab_list

def generate_stabilizer_elements(generators: List[cirq.PauliString]) -> List[cirq.PauliString]:
    elements = []
    for string in itertools.chain.from_iterable(itertools.combinations(generators, r) for r in range(len(generators) + 1)):
        elements.append(
            functools.reduce(lambda a, b: a * b, string, cirq.PauliString())
        )
    return elements

# For qiskit circuits
def get_active_qubits(circ):
    dag = qiskit.converters.circuit_to_dag(circ)
    active_qubits = [qubit for qubit in circ.qubits if qubit not in dag.idle_wires()]
    return active_qubits

def get_lst_ev(counts, observables, stabilizers):
    numerator = 0
    for obs in observables:
        numerator += compute_expectation(obs, counts) / len(observables)
    denominator = 0
    for stab in stabilizers:
        denominator += compute_expectation(stab, counts) / len(stabilizers)
    return float(np.real_if_close(numerator / denominator))


### Run unmitigated experiment

In [17]:
qreg = cirq.LineQubit.range(k)

circuit = cirq.Circuit()
circuit.append(cirq.H.on(qreg[0]))
for i in range(len(qreg)-1):
    circuit.append(cirq.CNOT.on(qreg[i], qreg[i+1]))
circuit.append(cirq.H.on_each(qreg))

print(circuit)

circuit = qiskit.QuantumCircuit.from_qasm_str(circuit.to_qasm())
circuit.measure_active()
# Compile to device.
compiled_raw = qiskit.transpile(
    circuit, 
    backend=computer,
    initial_layout=layout[1],  # Hardcode n = 1 (i.e., no encoding) to get layout.
    routing_method="sabre",
    # scheduling_method="asap",
    optimization_level=0,
)

job = computer.run(compiled_raw, shots=nshots)
counts = job.result().get_counts()
ev = compute_expectation(PauliString("XX")._pauli, counts)

print(ev)

0: ───H───@───H───
          │
1: ───────X───H───
1.0


## Run encoded experiment

In [18]:
generator_strs = [
    "XXXXIII",
    "IXXIXXI",
    "IIXXIXX",
    "ZZZZIII",
    "IZZIZZI",
    "IIZZIZZ"
]

observable = PauliString("X" * n * k)._pauli

qreg = cirq.LineQubit.range(n * k)

stabilizer_generators = strs_to_paulis(generator_strs)


stabilizer_matrix = get_stabilizer_matrix_from_paulis(stabilizer_generators, qreg[:n])
meas_circuit, transformed_matrix = get_measurement_circuit(stabilizer_matrix)

"""
Warning: Here you can insert like this the hadamard gate (logical) as the stabilizer group is non affected by this transformation. If you were going to apply it to other code you should always check that HS = S (generally work but fake hadamard gate e.g repetition code hadamard does not work).
"""

meas_circuit.insert(0, cirq.H.on_each(qreg[:n]))

measurement_circuit = measurement_circuit = cirq.Circuit.concat_ragged(
    *[meas_circuit,meas_circuit.transform_qubits(dict(zip(qreg[:n], qreg[n:2*n])))]
)

transformed_generators = get_paulis_from_stabilizer_matrix(transformed_matrix)[:-1] 
transformed_generators = [transformed_generators,transformed_generators]

stabilizer_elements = generate_stabilizer_elements(
    functools.reduce(lambda a,b : a+b, [
        [p.map_qubits(dict(zip(qreg[:n], qreg[i*n:(i+1)*n]))) for p in transformed_generators[i]] for i in range(k)
    ])
)

transformed_observable = observable.conjugated_by(measurement_circuit**-1)
observable_elements = [transformed_observable * stab for stab in stabilizer_elements]

In [19]:
encoding = cirq.Circuit.concat_ragged(
    encode_steane(qreg[:n]),
    encode_steane(qreg[n:])
)
encoding.append(cirq.H.on_each(qreg[:n]))
encoding.append(cirq.CNOT.on_each([(qreg[i], qreg[i+n]) for i in range(n)]))


tot_circuit = encoding+ measurement_circuit
tot_circuit

┌──┐   ┌──┐   ┌──┐   ┌───┐   ┌────┐                                       ┌───┐   ┌───┐   ┌──┐
0: ────H────@───────@─────H───────@──────────────H───@───@───────────────────────@──────@─────H─────────────────
            │       │             │                  │   │                       │      │
1: ─────────X───────┼─────X──────H┼──────@───────H───X───┼───@───@───────────────┼─────@┼──────@──────H─────────
                    │     │       │      │               │   │   │               │     ││      │
2: ────────────────X┼─────┼X─────H┼──────┼@──────H───────X───X───┼───────────────┼────@┼┼─────@┼──────@─────H───
                   ││     ││      │      ││                      │               │    │││     ││      │
3: ──────────X─────┼X─────┼@─────H┼──────┼┼@─────H───H───────────┼───────────X───@────@┼┼─────┼┼H─────┼─────────
             │     │      │       │      │││                     │           │         ││     ││      │
4: ────H────@┼─────@──────@──────H┼──────┼┼┼@────H───────────────X───×───────┼───X─────@┼─────@┼──────┼H────────
            ││                    │      ││││                        │       │   │      │      │      │
5: ─────────X┼─────X──────H──────@┼──────┼┼┼┼────H───H───────────────×───×───@───@──────@──────@──────@─────H───
             │     │             ││      ││││                            │
6: ────H─────@─────@──────H──────┼┼@─────┼┼┼┼────H───H───────────────────×──────────────────────────────────────
                                 │││     ││││
7: ────H────@───────@────────────┼X┼─────┼┼┼┼────H───@───@───────────────────────@──────@─────H─────────────────
            │       │            │ │     ││││        │   │                       │      │
8: ─────────X───────┼─────X──────┼─┼─────X┼┼┼────H───X───┼───@───@───────────────┼─────@┼──────@──────H─────────
                    │     │      │ │      │││            │   │   │               │     ││      │
9: ────────────────X┼─────┼X─────┼─┼──────X┼┼────H───────X───X───┼───────────────┼────@┼┼─────@┼──────@─────H───
                   ││     ││     │ │       ││                    │               │    │││     ││      │
10: ─────────X─────┼X─────┼@─────┼─┼───────X┼────H───H───────────┼───────────X───@────@┼┼─────┼┼H─────┼─────────
             │     │      │      │ │        │                    │           │         ││     ││      │
11: ───H────@┼─────@──────@──────┼─┼────────X────H───────────────X───×───────┼───X─────@┼─────@┼──────┼H────────
            ││                   │ │                                 │       │   │      │      │      │
12: ────────X┼─────X─────────────X─┼─────────────H───H───────────────×───×───@───@──────@──────@──────@─────H───
             │     │               │                                     │
13: ───H─────@─────@───────────────X─────────────H───H───────────────────×──────────────────────────────────────
           └──┘   └──┘   └──┘   └───┘   └────┘                                       └───┘   └───┘   └──┘

In [21]:
encoding = cirq.Circuit.concat_ragged(
    encode_steane(qreg[:n]),
    encode_steane(qreg[n:])
)

# prepare Bell state
encoding.append(cirq.H.on_each(qreg[:n]))
encoding.append(cirq.CNOT.on_each([(qreg[i], qreg[i+n]) for i in range(n)]))

encoding = qiskit.QuantumCircuit.from_qasm_str(encoding.to_qasm())
measurement = qiskit.QuantumCircuit.from_qasm_str(measurement_circuit.to_qasm())

circ_full = encoding.compose(
    Barrier(n*k, label="encoding"), get_active_qubits(encoding)
).compose(
    Barrier(n*k, label="measurement"), get_active_qubits(encoding)
).compose(measurement)
circ_full.measure_active()

compiled = qiskit.transpile(
    circ_full, 
    backend=computer,
    # initial_layout=layout[n],
    routing_method="sabre",
    # scheduling_method="asap",
    optimization_level=3,
)

compiled.draw(fold=-1, idle_wires=False)
compiled.count_ops()

OrderedDict([('cx', 35),
             ('h', 33),
             ('cz', 14),
             ('measure', 14),
             ('barrier', 3)])

In [22]:
job = computer.run(
    compiled,
    shots=nshots,
)

counts = job.result().get_counts()
print(counts)
ev = get_lst_ev(counts, tqdm(observable_elements), tqdm(stabilizer_elements))
print(ev)

{'00000000000000': 50052, '10100001010000': 49948}


100%|██████████| 1024/1024 [00:00<00:00, 37357.29it/s]

1.0
